In [1]:
SCHOOL="minnesota"
SUBJECT="opinion"
START_YEAR=2017
FINAL_YEAR=2022
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [2]:
cd /Users/anishka/Desktop/asdrp/mui/diversity-colleges/diversity-colleges/

/Users/anishka/Desktop/asdrp/mui/diversity-colleges/diversity-colleges


In [3]:
import sys
sys.path.append('src')
import ouraws
import ourgraphs
#import textutil

In [4]:
S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [5]:
df.shape

(1670, 6)

In [6]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API")

BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [7]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

-13.261775


In [8]:
df['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010, 2009, 2008])

In [9]:
results = []

for year in range(2020, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2020 ... 237
2020	237	-928.7379080000003	-3.9187253502109716
Year: 2021 ... 175
2021	175	-359.97684299999975	-2.05701053142857
Year: 2022 ... 174
2022	174	-304.09456899999986	-1.7476699367816084
